## Clean and extract new data from raw monthly datasets.

### Contents
1. Data cleaning
2. Generate the revenue per trip.
3. Get public holidays.
4. Get and join weather data.
5. Export data for analysis.

In [13]:
# Import libraries
import pandas as pd # for data manipulation and analysis
import matplotlib.pyplot as plt # for plotting
import re # In-built regular expressions library

In [14]:
# Import all the monthly csv dataset

dec_20 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202012-divvy-tripdata.csv')
jan_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202101-divvy-tripdata.csv')
feb_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202102-divvy-tripdata.csv')
march_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202103-divvy-tripdata.csv')
april_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202104-divvy-tripdata.csv')
may_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202105-divvy-tripdata.csv')
jun_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202106-divvy-tripdata.csv')
july_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202107-divvy-tripdata.csv')
aug_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202108-divvy-tripdata.csv')
sept_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202109-divvy-tripdata.csv')
oct_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202110-divvy-tripdata.csv')
nov_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Cyclistic Project/cyclistic project/202111-divvy-tripdata.csv')


# Join all months into a single dataframe
months = [dec_20, jan_21, feb_21, march_21, april_21, may_21, jun_21, july_21, aug_21, sept_21, oct_21, nov_21]
df_months = pd.concat(months)

# Check the number of data points
print(f'Number of data points: {df_months.shape[0]}')


Number of data points: 5479096


In [15]:
# Drop duplicates
df = df_months
print('Number of duplicate ride_ids:',df['ride_id'].duplicated().sum())

df = df.drop_duplicates(subset=['ride_id'])
print('No. duplicates after dropping:', df['ride_id'].duplicated().sum())

Number of duplicate ride_ids: 0
No. duplicates after dropping: 0


In [16]:
# Handle null rows

# Drop rows without start and end time information
df = df.dropna(subset= ['started_at','ended_at'])

df = df.sort_values(by=['start_lat','start_lng'], ascending=True)

# Forward fill the missing start_station_name and start_station_id
df[['start_station_name', 'start_station_id']] = df[['start_station_name', 'start_station_id']].ffill().bfill()

# Fill the end stations
df = df.sort_values(by=['end_lat','end_lng'], ascending=True)
# Forward fill the missing start_station_name and start_station_id
df[['end_station_name', 'end_station_id','end_lat','end_lng']] = df[['end_station_name', 'end_station_id','end_lat','end_lng']].ffill().bfill()

df.isna().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

In [17]:
# Data Extraction
df_clean = df

# Funtion to preprocess datetime string, adding seconds if missing
def preprocess_datetime(dt_str):
    if pd.isna(dt_str):
        return np.nan
    if len(dt_str) == 16:  # Format is missing seconds (YYYY-MM-DD HH:MM)
        return dt_str + ':00'
    # Add leading zero to single-digit hours
    if len(dt_str) == 15 and re.match(r'\d{4}-\d{2}-\d{2} \d:\d{2}', dt_str):
        return dt_str[:11] + '0' + dt_str[11:] + ':00'
    
    # Add leading zero to single-digit hours with missing seconds
    if len(dt_str) == 14 and re.match(r'\d{4}-\d{2}-\d{2} \d:\d{2}', dt_str):
        return dt_str[:11] + '0' + dt_str[11:] + ':00'
    return dt_str

# Apply the preprocessing function to both columns
df_clean['started_at'] = df_clean['started_at'].apply(preprocess_datetime)
df_clean['ended_at'] = df_clean['ended_at'].apply(preprocess_datetime)

# Convert to date time
df_clean['started_at'] = pd.to_datetime(df_clean['started_at'],format='%Y-%m-%d %H:%M:%S')
df_clean['ended_at'] = pd.to_datetime(df_clean['ended_at'],format='%Y-%m-%d %H:%M:%S')
# Extract the month and day of the week
df['day'] = df['started_at'].dt.day_name()
df['month'] = df['started_at'].dt.strftime("%B")

# Find the ride duration in seconds
df_clean['ride_duration'] = (df_clean['ended_at']-df_clean['started_at']).dt.total_seconds()

In [18]:
#Remove outliers

p25 = df_clean['ride_duration'].quantile(0.25) # 25th percentile
p50 = df_clean['ride_duration'].quantile(0.5) # 50th percentile
p75 = df_clean['ride_duration'].quantile(0.75) # 75th percentile

# Interquartile range
iqr = p75 - p25
print(f'Interquartile range: {iqr}')

# Lower Limit
lower_bound  = p25 - 1.5 * iqr
print(f'Lower Bound: {lower_bound}')

# Upper Limit
upper_bound = p75 + 1.5 * iqr
print(f'Upper Bound: {upper_bound}')

# Mean
print(f'Mean: {df_clean['ride_duration'].mean()}')

# Eliminate outliers above the upper bound and negative ride durations
df_clean = df_clean[(df_clean['ride_duration'] >= 0) & (df_clean['ride_duration'] <= upper_bound)]

print(f'No of data points after cleaning: {df_clean.shape[0]}')

Interquartile range: 910.0
Lower Bound: -955.0
Upper Bound: 2685.0
Mean: 1209.2180638922916
No of data points after cleaning: 5072961


### Processing and Extraction

In [ ]:
# Extract the hour of the day
df_clean['hour'] = df_clean['started_at'].dt.hour # extract the ride hour

# function to assign time of day
def time_of_day(row):
    if 0 <= row['hour'] < 6:
        return 'Early Morning'
    elif 6 <= row['hour'] < 12:
        return 'Morning'
    elif 12 <= row['hour'] < 18:
        return 'Afternoon'
    elif 18 <= row['hour'] < 21:
        return 'Evening'
    elif 21 <= row['hour'] < 24:
        return 'Night'
    
df_clean['time_of_day'] = df_clean.apply(time_of_day, axis = 1)

In [ ]:
# Function to assign seasons based on the month
def assign_season(row):
    if row['month'] in ['December', 'January', 'February']:
        return 'Winter'
    elif row['month'] in ['March', 'April', 'May']:
        return 'Spring'
    elif row['month'] in ['June', 'July', 'August']:
        return 'Summer'
    elif row['month'] in ['September', 'October', 'November']:
        return 'Fall'
    
# Apply the function to assign seasons
df_clean['season'] = df_clean.apply(assign_season, axis=1)

### 2. Generate the revenue per trip
For the revenue generated for each trip, I adopted a similar pricing schema used by a local bike sharing company, BIKE SHARE TORONTO. The pricing goes as follow:
- Members:
    - Unlocking Fee = $0
    - Classic Bike  = $0.17/minute
    - Electric Bike = $0.17/minute
    - Docked Bike = $0.20/minute

- Casual:
    - Unlocking Fee = $1.35
    - Classic Bike  = $0.20/minute
    - Electric Bike = $0.35/minute
    - Docked Bike = $0.40/minute


In [ ]:
def add_revenue(row):

    ride_duration_mins = row['ride_duration']/60 # convert to minutes

    if row['member_casual'] == 'casual':

        unlocking_fee = 1.35 # Casual unlocking fee
        if row['rideable_type'] == 'electric_bike':
            return unlocking_fee + 0.35 * ride_duration_mins
        elif row['rideable_type'] == 'classic_bike':
            return unlocking_fee + 0.20 * ride_duration_mins
        elif row['rideable_type'] == 'docked_bike':
            return unlocking_fee + 0.40 * ride_duration_mins
        else:
            return 0
        
    elif row['member_casual'] == 'member':
        if row['rideable_type'] == 'electric_bike':
            return 0.17 * ride_duration_mins
        elif row['rideable_type'] == 'classic_bike':
            return 0.17 * ride_duration_mins
        elif row['rideable_type'] == 'docked_bike':
            return 0.20 * ride_duration_mins
        else:
            return 0
    else:
        return None

df_clean['revenue'] = df_clean.apply(add_revenue, axis = 1)

### 3. Get public holidays
Here's my approach to achieve this:
1. Get all public holidays in Chicago that year (2021) via web scraping
2. Extract the date from the 'started_at' column
3. Join with the main dataframe

In [ ]:
url = 'https://www.timeanddate.com/holidays/us/2021'

# Scrape the raw table
holiday_data = pd.read_html(url)[0]

# Drop first levl column of the multi-index
holiday_data.columns = holiday_data.columns.droplevel(0)

#renam column
df_holiday = holiday_data.rename(columns={'Unnamed: 1_level_1': 'Day'})

# Select needed columns and rows
holidays = df_holiday[['Date','Day','Name']].iloc[1:-1].reset_index(drop=True)

# Extract the date

df_clean['start_date'] = df_clean['started_at'].dt.strftime("%b %d")
df_clean['start_day'] = df_clean['started_at'].dt.strftime("%Y-%m-%d")

# Join 
holiday_df = pd.merge(df_clean, holidays, left_on ='start_date', right_on = 'Date', how = 'left').drop_duplicates(subset = 'ride_id')

# get holiday columns
def holiday(row):
    if pd.isna(row['Name']):
        return 'Not Holiday'
    else:
        return 'Holiday'
    
holiday_df['holiday'] = holiday_df.apply(holiday, axis = 1)

### 4. Get and Join the daily weather

In [23]:
# import climate data
weather_df = pd.read_csv(r'./Chicago,United States 2020-12-01 to 2021-12-01.csv')

# Clean data
# Convert the 'datetime' column to actual datetime objects
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'], format='%Y-%m-%dT%H:%M:%S')
weather_df['datetime'] = weather_df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_weather = weather_df[['datetime','conditions','icon','temp','humidity','windspeed']]

holiday_df['start_date_hour'] = holiday_df['started_at'].apply(lambda x: x.replace(minute=0, second=0))
# Optionally convert it back to string format
holiday_df['start_date_hour'] = holiday_df['start_date_hour'].dt.strftime("%Y-%m-%d %H:%M:%S")

full_df = pd.merge(holiday_df, df_weather, left_on='start_date_hour', right_on = 'datetime', how ='left').drop_duplicates(subset = 'ride_id')

In [24]:
# function to group the temperature into  (warm, cold, hot, cool)
def categorize_temperature(row):
    if row['temp'] < 10:
        return 'Cold'
    elif 10 <= row['temp'] < 20:
        return 'Cool'
    elif 20 <= row['temp'] < 30:
        return 'Warm'
    else:
        return 'Hot'
    
# Apply the function
full_df['temp_feel'] = full_df.apply(categorize_temperature, axis = 1)

# Preview data
full_df.head(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,Name,holiday,start_date_hour,datetime,conditions,icon,temp,humidity,windspeed,temp_feel
0,097BB436A70E5964,electric_bike,2021-02-27 01:27:00,2021-02-27 02:09:00,Indiana Ave & 133rd St,20240,Hegewisch Metra Station,20215,41.65,-87.54,...,NaN,Not Holiday,2021-02-27 01:00:00,2021-02-27 01:00:00,"Rain, Overcast",rain,2.4,81.47,10.1,Cold
1,401BC56B23FA4A26,electric_bike,2021-10-08 12:12:05,2021-10-08 12:35:25,Major Taylor Trail & 124th St,20208,Hegewisch Metra Station,20215,41.67,-87.55,...,NaN,Not Holiday,2021-10-08 12:00:00,2021-10-08 12:00:00,Partially cloudy,partly-cloudy-day,22.0,66.12,11.0,Warm


Export data

In [25]:
full_df.to_csv(r'./Cleaned Data.csv', index=False)

Sources:
- Weather: 'https://www.visualcrossing.com/weather/weather-data-services/Chicago,United%20States/metric/2020-12-01/2021-11-30'
- Holidays: 'https://www.timeanddate.com/weather/usa/chicago/historic?month=1&year=2021'